In [66]:
columns = ["Content", "Important"]
train = pd.concat([
    pd.read_excel("etools/emails_300_set_1.xlsx", header=1)[columns],
    pd.read_excel("etools/emails_300_set_2.xlsx", header=1)[columns],
    pd.read_excel("etools/emails_300_set_3.xlsx", header=1)[columns],  
])

train.to_csv("train.csv")

In [2]:
import importlib

In [69]:
import etools.ranker as erank
import etools.summarizer as esum

In [70]:
%%time
importlib.reload(erank)
importlib.reload(esum)

Wall time: 7.66 s


In [29]:
scores = np.array([erank.score(email) for email in test])

In [71]:
for i, email in enumerate(test):
    a = erank.is_important(email)
    if not a:
        print(i)

269
418


In [20]:
email = test[1]
for i, s in enumerate(esum.significant_sentences(email)):
    print("{}.)".format(i + 1), s)
    
for i, s in enumerate(esum.summarizing_sentences(email)):
    print("{}.)".format(i + 1), s)

1.) a. I have made most of the changes suggested in Gayle's fax of 11/29 except that the definition of Performance Assurance needs to stay as it is given that it does not only mean the Escrowed Funds (for instance, in the event of Downgrade Event with respect to Enron Corp., EPMI would have to post Performance Assurance).
2.) We have also not amended the last line of Section 5.5 since, among other things, the two amendments taken together would mean that in the event of a dispute, the Non-Defaulting Party would be limited to receiving only what is in the Escrow Fund.
1.) I will be happy to incorporate the Escrow language in the La Salle Bank Escrow Agreement once someone from Black Hills lets me know that the Escrow language is acceptable as written.
2.) a. I have made most of the changes suggested in Gayle's fax of 11/29 except that the definition of Performance Assurance needs to stay as it is given that it does not only mean the Escrowed Funds (for instance, in the event of Downgrad

# Summarizer Scratch

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
import spacy
import pickle

## Significant Word Finder

In [5]:
%%time
# Load in the training and test data.
train = pd.read_excel("../Data/emails_25000.xlsx", header=1)["Content"]
test  = pd.concat([
    pd.read_excel("../Data/emails_300_set_1.xlsx", header=1),
    pd.read_excel("../Data/emails_300_set_2.xlsx", header=1),
    pd.read_excel("../Data/emails_300_set_3.xlsx", header=1),
]).reset_index()["Content"]

Wall time: 4.8 s


In [ ]:
def clean(document: str) -> str:
    document = document.lower()
    words = "".join(c for c in document if c.isalpha() or c.isspace()).split()
    return " ".join(w for w in words if w not in ENGLISH_STOP_WORDS)

clean("Hello. My name is Daniel")

In [ ]:
%%time
# Train a TF-IDF Vectorizer.
tfidf = TfidfVectorizer(
    ngram_range=(1, 1),
    token_pattern="[A-Za-z]{3,}"  # Letter-only words of length at least 3.
).fit(clean(email) for email in train)

In [ ]:
with open("tfidf.pkl", "wb") as file:
    pickle.dump(tfidf, file)

In [ ]:
def get_significant_words(doc: str, vectorizer, num_words: int = 5):
    """ Get the *num_words* most significant words from this *doc*. """
    doc = clean(doc)
    vector = vectorizer.transform([doc]).toarray().flatten()
    words = sorted(list(zip(vector, vectorizer.get_feature_names())))
    try:
        return [word for _, word in words[-num_words:]][::-1]
    except IndexError as error:
        print("Error: num_words parameter greater than number of words.")
        return []
    
get_significant_words(test[20], tfidf)

## Sentence Summarizer

In [ ]:
nlp = spacy.load("en_core_web_md")

In [ ]:
def get_significant_sentence(doc: str, vectorizer, num_sentences: int = 1):
    sentences = [sentence.string.strip() for sentence in nlp(doc).sents]
    scores = [vectorizer.transform([clean(s)]).sum() for s in sentences]
    pairs = [(scores[i], sentences[i]) for i in range(len(sentences))]
    pairs = sorted(pairs)[::-1]
    try:
        return [pair[1] for pair in pairs[:num_sentences]]
    except IndexError as error:
        print("Error: num_words parameter greater than number of words.")
        return []   
    
email = test[20]
sentences = get_significant_sentence(email, tfidf, num_sentences=2)
for i, sentence in enumerate(sentences):
    print("{}.) ".format(i + 1), " ".join(sentence.split()))    
print("\n=====================================\n")
print(email)

In [ ]:
def get_summarizing_sentence(doc: str, num_sentences: int = 1):
    counts = {}
    for word in nlp(clean(doc)):
        counts[word.lemma_] = counts.get(word.lemma_, -1) + 1
    
    text = nlp(doc)
    sentences = [sentence.string.strip() for sentence in text.sents]
    scores = [0 for _ in sentences]
    for i, sentence in enumerate(text.sents):
        for word in nlp(clean(sentence.string)):
            scores[i] += counts.get(word.lemma_, 0)
                
    pairs = [(scores[i], sentences[i]) for i in range(len(sentences))]
    pairs = sorted(pairs)[::-1]
    
    try:
        return [pair[1] for pair in pairs[:num_sentences]]
    except IndexError as error:
        print("Error: num_words parameter greater than number of words.")
        return []  
    
email = test[20]
sentences = get_summarizing_sentence(email, num_sentences=2)
for i, sentence in enumerate(sentences):
    print("{}.) ".format(i + 1), " ".join(sentence.split()))    
print("\n=====================================\n")
print(email)